In [260]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib
from datetime import timedelta
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from pandas import factorize

%matplotlib inline

### AFSNT__DATA요약
통계량 및 집계


In [261]:
data=pd.read_csv('./bigcon2019/AFSNT.csv',encoding='CP949',header=1,
                names=['연','월','일','요일','공항','상대공항','항공사','편명','등록기호','출도착',\
                      '부정기타입','계획시각','실제시각','지연여부','지연사유','결항여부','결항사유'])



C:\Users\vldya\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Column names 변환 , + IRR  : 부정기 타입

In [262]:
data.head()

,연,월,일,요일,공항,상대공항,항공사,편명,등록기호,출도착,부정기타입,계획시각,실제시각,지연여부,지연사유,결항여부,결항사유
0,2017,1,1,일,ARP1,ARP3,A,A1905,SEw4MjM2,D,N,6:15,6:25,N,NaN,N,NaN
1,2017,1,1,일,ARP1,ARP3,L,L1751,SEw4MjM3,D,N,6:20,6:30,N,NaN,N,NaN
2,2017,1,1,일,ARP1,ARP3,F,F1201,SEw4MjA3,D,N,6:25,6:34,N,NaN,N,NaN
3,2017,1,1,일,ARP3,ARP1,A,A1900,SEw3NzAz,D,N,6:30,6:37,N,NaN,N,NaN
4,2017,1,1,일,ARP1,ARP3,H,H1101,SEw4MDMx,D,N,6:30,6:38,N,NaN,N,NaN


In [263]:
data.shape

(987708, 17)

In [264]:
print(data.count())
print('-'*50)
print(data.groupby('연')['결항여부'].count())
print('-'*50)
print(data.groupby('연')['결항사유'].count())

연        987708
월        987708
일        987708
요일       987708
공항       987708
상대공항     987708
항공사      987708
편명       987708
등록기호     987250
출도착      987708
부정기타입    987708
계획시각     987708
실제시각     979460
지연여부     987708
지연사유     119917
결항여부     987708
결항사유       3018
dtype: int64
--------------------------------------------------
연
2017    397511
2018    395925
2019    194272
Name: 결항여부, dtype: int64
--------------------------------------------------
연
2017    1868
2018       0
2019    1150
Name: 결항사유, dtype: int64


In [265]:
print(data.groupby('지연여부')['연'].count())

지연여부
N    868771
Y    118937
Name: 연, dtype: int64


### 항공기 / 편명 탐색

In [266]:
print("편명 개수:", data.편명.unique().size)
print("항공사 개수:", data.항공사.unique().size)
print("등록기호 개수:", data.등록기호.unique().size)


편명 개수: 1541
항공사 개수: 12
등록기호 개수: 365


In [267]:
#월별 운행 빈도수 출력
print(data.groupby('월')['출도착'].count())

월
1      96847
2      88259
3      98396
4      98868
5     103473
6     100182
7      68481
8      68671
9      67244
10     68471
11     63706
12     65110
Name: 출도착, dtype: int64


## 항공사별 등록기호 카운트

In [269]:
# 항공사별 등록기호별 개수 출력
Enroll_by_Air = data.pivot_table(index= ['항공사','등록기호'],
                aggfunc='count',
                values='지연여부')
Enroll_by_Air

지연여부
항공사 등록기호           
A   SEw3MjQ3  10590
    SEw3MjQ4   2276
    SEw3NDE4     15
    SEw3NTA2  11035
    SEw3NTE0  11535
    SEw3NTE1    275
    SEw3NTE2   1983
    SEw3NTI4   6984
    SEw3NTk0  15281
    SEw3NTk3      8
    SEw3NzAw      3
    SEw3NzAz  15194
    SEw3NzM2    245
    SEw3NzM3   9906
    SEw3NzM4  10078
    SEw3NzM5     10
    SEw3NzMy      6
    SEw3NzQ2     31
    SEw3NzQ3    477
    SEw3NzQw    286
    SEw3NzQx    245
    SEw3NzU0    442
    SEw3NzU1     16
    SEw3NzU2     17
    SEw3NzY5  11028
    SEw3NzYz    604
    SEw3Nzc1      7
    SEw3Nzc2  10250
    SEw3Nzcy   8644
    SEw3Nzcz  10134
...             ...
K   QjUwMDE2      2
L   SEw4MDAw   2745
    SEw4MDI0   3014
    SEw4MDIx   2900
    SEw4MDMw   2800
    SEw4MDQ3   4870
    SEw4MDU2   4927
    SEw4MDY3   4490
    SEw4MDY5   5276
    SEw4MDcw   4864
    SEw4MDg2   3083
    SEw4MDk1   3426
    SEw4MDk4   3085
    SEw4MjIw   1296
    SEw4MjM1   8668
    SEw4MjM3   6165
    SEw4MjMy   1070
    SEw4MjUz   4347
    SEw4MjY4   5568
    SEw4Mjk0   2863
    SEw4MzA2   1567
    SEw4MzAw   1594
    SEw4MzI0    896
    SEw4MzI2    802
    SEw4MzI3    815
    SEw4MzI5    675
    SEw4MzIz    946
    SEw4MzU0    694
    SEw4MzYz    242
    SExYMDE=      1

[372 rows x 1 columns]

In [270]:
# 공항수 카운트
print("공항 개수:", data.공항.unique().size)
print("상대공항 개수:", data.상대공항.unique().size)

공항 개수: 15
상대공항 개수: 15


#### 날짜 변수 결합

In [271]:
data['date']=pd.to_datetime(data['연'].apply(str)+'-'+
                            data['월'].apply(str)+'-'+
                            data['일'].apply(str))

In [272]:
data

,연,월,일,요일,공항,상대공항,항공사,편명,등록기호,출도착,부정기타입,계획시각,실제시각,지연여부,지연사유,결항여부,결항사유,date
0,2017,1,1,일,ARP1,ARP3,A,A1905,SEw4MjM2,D,N,6:15,6:25,N,NaN,N,NaN,2017-01-01
1,2017,1,1,일,ARP1,ARP3,L,L1751,SEw4MjM3,D,N,6:20,6:30,N,NaN,N,NaN,2017-01-01
2,2017,1,1,일,ARP1,ARP3,F,F1201,SEw4MjA3,D,N,6:25,6:34,N,NaN,N,NaN,2017-01-01
3,2017,1,1,일,ARP3,ARP1,A,A1900,SEw3NzAz,D,N,6:30,6:37,N,NaN,N,NaN,2017-01-01
4,2017,1,1,일,ARP1,ARP3,H,H1101,SEw4MDMx,D,N,6:30,6:38,N,NaN,N,NaN,2017-01-01
5,2017,1,1,일,ARP1,ARP3,I,I1301,SEw3NTYz,D,N,6:30,6:44,N,NaN,N,NaN,2017-01-01
6,2017,1,1,일,ARP1,ARP3,A,A1907,SEw3NTk0,D,N,6:40,7:54,Y,C01,N,NaN,2017-01-01
7,2017,1,1,일,ARP1,ARP3,I,I1303,SEw4MDE1,D,N,6:40,6:53,N,NaN,N,NaN,2017-01-01
8,2017,1,1,일,ARP1,ARP3,J,J1204,SEw3Nzg1,D,N,6:45,7:02,N,NaN,N,NaN,2017-01-01
9,2017,1,1,일,ARP1,ARP3,A,A1903,SEw3Nzcz,D,N,6:45,6:55,N,NaN,N,NaN,2017-01-01


### 불필요한 Column 삭제


In [273]:
data.drop(['연','월','일'],axis=1,inplace=True)

In [274]:
data

,요일,공항,상대공항,항공사,편명,등록기호,출도착,부정기타입,계획시각,실제시각,지연여부,지연사유,결항여부,결항사유,date
0,일,ARP1,ARP3,A,A1905,SEw4MjM2,D,N,6:15,6:25,N,NaN,N,NaN,2017-01-01
1,일,ARP1,ARP3,L,L1751,SEw4MjM3,D,N,6:20,6:30,N,NaN,N,NaN,2017-01-01
2,일,ARP1,ARP3,F,F1201,SEw4MjA3,D,N,6:25,6:34,N,NaN,N,NaN,2017-01-01
3,일,ARP3,ARP1,A,A1900,SEw3NzAz,D,N,6:30,6:37,N,NaN,N,NaN,2017-01-01
4,일,ARP1,ARP3,H,H1101,SEw4MDMx,D,N,6:30,6:38,N,NaN,N,NaN,2017-01-01
5,일,ARP1,ARP3,I,I1301,SEw3NTYz,D,N,6:30,6:44,N,NaN,N,NaN,2017-01-01
6,일,ARP1,ARP3,A,A1907,SEw3NTk0,D,N,6:40,7:54,Y,C01,N,NaN,2017-01-01
7,일,ARP1,ARP3,I,I1303,SEw4MDE1,D,N,6:40,6:53,N,NaN,N,NaN,2017-01-01
8,일,ARP1,ARP3,J,J1204,SEw3Nzg1,D,N,6:45,7:02,N,NaN,N,NaN,2017-01-01
9,일,ARP1,ARP3,A,A1903,SEw3Nzcz,D,N,6:45,6:55,N,NaN,N,NaN,2017-01-01


In [278]:
#요일 변경 

data['wday'] = data.요일.map({
    '일': 0,
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
    '토': 6
})
data

,요일,공항,상대공항,항공사,편명,등록기호,출도착,부정기타입,계획시각,실제시각,지연여부,지연사유,결항여부,결항사유,date,wday
0,일,ARP1,ARP3,A,A1905,SEw4MjM2,D,N,6:15,6:25,N,NaN,N,NaN,2017-01-01,0
1,일,ARP1,ARP3,L,L1751,SEw4MjM3,D,N,6:20,6:30,N,NaN,N,NaN,2017-01-01,0
2,일,ARP1,ARP3,F,F1201,SEw4MjA3,D,N,6:25,6:34,N,NaN,N,NaN,2017-01-01,0
3,일,ARP3,ARP1,A,A1900,SEw3NzAz,D,N,6:30,6:37,N,NaN,N,NaN,2017-01-01,0
4,일,ARP1,ARP3,H,H1101,SEw4MDMx,D,N,6:30,6:38,N,NaN,N,NaN,2017-01-01,0
5,일,ARP1,ARP3,I,I1301,SEw3NTYz,D,N,6:30,6:44,N,NaN,N,NaN,2017-01-01,0
6,일,ARP1,ARP3,A,A1907,SEw3NTk0,D,N,6:40,7:54,Y,C01,N,NaN,2017-01-01,0
7,일,ARP1,ARP3,I,I1303,SEw4MDE1,D,N,6:40,6:53,N,NaN,N,NaN,2017-01-01,0
8,일,ARP1,ARP3,J,J1204,SEw3Nzg1,D,N,6:45,7:02,N,NaN,N,NaN,2017-01-01,0
9,일,ARP1,ARP3,A,A1903,SEw3Nzcz,D,N,6:45,6:55,N,NaN,N,NaN,2017-01-01,0


#### 컬럼순서 바꾸기

In [282]:
air_df = pd.DataFrame(data, columns=['date','wday','요일','공항','상대공항','항공사','편명','등록기호','출도착',\
                      '부정기타입','계획시각','실제시각','지연여부','지연사유','결항여부','결항사유'])

In [283]:
air_df

,date,wday,요일,공항,상대공항,항공사,편명,등록기호,출도착,부정기타입,계획시각,실제시각,지연여부,지연사유,결항여부,결항사유
0,2017-01-01,0,일,ARP1,ARP3,A,A1905,SEw4MjM2,D,N,6:15,6:25,N,NaN,N,NaN
1,2017-01-01,0,일,ARP1,ARP3,L,L1751,SEw4MjM3,D,N,6:20,6:30,N,NaN,N,NaN
2,2017-01-01,0,일,ARP1,ARP3,F,F1201,SEw4MjA3,D,N,6:25,6:34,N,NaN,N,NaN
3,2017-01-01,0,일,ARP3,ARP1,A,A1900,SEw3NzAz,D,N,6:30,6:37,N,NaN,N,NaN
4,2017-01-01,0,일,ARP1,ARP3,H,H1101,SEw4MDMx,D,N,6:30,6:38,N,NaN,N,NaN
5,2017-01-01,0,일,ARP1,ARP3,I,I1301,SEw3NTYz,D,N,6:30,6:44,N,NaN,N,NaN
6,2017-01-01,0,일,ARP1,ARP3,A,A1907,SEw3NTk0,D,N,6:40,7:54,Y,C01,N,NaN
7,2017-01-01,0,일,ARP1,ARP3,I,I1303,SEw4MDE1,D,N,6:40,6:53,N,NaN,N,NaN
8,2017-01-01,0,일,ARP1,ARP3,J,J1204,SEw3Nzg1,D,N,6:45,7:02,N,NaN,N,NaN
9,2017-01-01,0,일,ARP1,ARP3,A,A1903,SEw3Nzcz,D,N,6:45,6:55,N,NaN,N,NaN


In [284]:
#날짜 색인 지정
air_df.set_index('date',inplace=True)

In [285]:
air_df2017=air_df['2017']
air_df2018=air_df['2018']
air_df2019=air_df['2019']

In [286]:
print("17년도 데이터 : ", air_df2017.size)
print("18년도 데이터 : ", air_df2018.size)
print("19년도 데이터 : ", air_df2019.size)

17년도 데이터 :  5962665
18년도 데이터 :  5938875
19년도 데이터 :  2914080


In [287]:
print("2017년", "-"*50)
for i in range(1,13):
    print("{}월 데이터 개수: ".format(i),air_df['2017-{}'.format(i)].size)
    
print("2018년", "-"*50)
for i in range(1,13):
    print("{}월 데이터 개수: ".format(i),air_df['2018-{}'.format(i)].size)
    
print("2019년", "-"*50)
for i in range(1,7):
    print("{}월 데이터 개수: ".format(i),air_df['2019-{}'.format(i)].size)

2017년 --------------------------------------------------
1월 데이터 개수:  485880
2월 데이터 개수:  440250
3월 데이터 개수:  496680
4월 데이터 개수:  503355
5월 데이터 개수:  519975
6월 데이터 개수:  501210
7월 데이터 개수:  514620
8월 데이터 개수:  513180
9월 데이터 개수:  505665
10월 데이터 개수:  520965
11월 데이터 개수:  475800
12월 데이터 개수:  485085
2018년 --------------------------------------------------
1월 데이터 개수:  481020
2월 데이터 개수:  441705
3월 데이터 개수:  491910
4월 데이터 개수:  493140
5월 데이터 개수:  518940
6월 데이터 개수:  502230
7월 데이터 개수:  512595
8월 데이터 개수:  516885
9월 데이터 개수:  502995
10월 데이터 개수:  506100
11월 데이터 개수:  479790
12월 데이터 개수:  491565
2019년 --------------------------------------------------
1월 데이터 개수:  485805
2월 데이터 개수:  441930
3월 데이터 개수:  487350
4월 데이터 개수:  486525
5월 데이터 개수:  513180
6월 데이터 개수:  499290


AttributeError: 'DataFrame' object has no attribute '요일'

In [310]:
#날짜 범위 검색
print(air_df.index.min(),"부터")
print(air_df.index.max(), "까지")
#weekday 확인
air_df.요일.value_counts()

2017-01-01 00:00:00 부터
2019-06-30 00:00:00 까지


일    143872
금    142842
월    141466
토    141237
목    139793
수    139320
화    139178
Name: 요일, dtype: int64

In [ ]:
#공항별 

In [312]:
import cx_Oracle


In [313]:
# Oracle DB 연결

conn_ora = cx_Oracle.connect("hr/hr@XE")

In [314]:
# Oracle DB 연결
# 접속정보(connection string) : ID/PASS@CONNECTION_ALIAS
# CONNECTION_ALIAS : Oracle TNSNAMES.ORA 파일에 있는 접속정보 별칭(ALIAS)
conn_ora = cx_Oracle.connect("hr/hr@xe")

# DB 커서(Cursor) 선언
cur = conn_ora.cursor()

# 사용할 Oracle 소스 테이블명 지정
src_table = "d_base3_2"

# 데이터프레임(rawData)에 저장된 데이터를 Oracle 테이블(d_base3_2)에 입력하기 위한 로직
# d_base3_2 테이블 존재하는지 체크하는 함수
def table_exists(name=None, con=None):
    sql = "select table_name from user_tables where table_name='MYTABLE'".replace('MYTABLE', name.upper())
    df = pd.read_sql(sql, con)

    # 테이블이 존재하면 True, 그렇지 않으면 False 반환
    exists = True if len(df) > 0 else False
    return exists

# 테이블(d_base3_2) 생성 (테이블이 이미 존재한다면 TRUNCATE TABLE)
if table_exists(src_table, conn_ora):
    cur.execute("TRUNCATE TABLE " + src_table)
else:
    cur.execute("create table " + src_table + " ( \
               Date varchar2(20), \
               wday varchar2(10),\
               요일 varchar2(10),\
               공항 varchar2(20),\
               상대공항 varchar2(20),\
               항공사 varchar2(20),\
               편명 varchar2(20),\
               등록기호 varchar2(100),\
               출도착 varchar2(10),\
               부정기타입 varchar2(10),\
               계획시각 varchar2(100),\
               실제시각 varchar2(100),\
               지연여부 varchar2(10),\
               지연사유 varchar2(20),\
               결항여부 varchar2(10),\
               결항사유 varchar2(20))")

# Sequence 구조를 Dictionary 구조((element, value))로 변환하는 함수
# 예: ("Matt", 1) -> {'1':'Matt', '2':1}
# INSERT INTO ... VALUES (:1, :2, ...) 에서 바인드 변수값을 주기위해 Dictionary item 구조 사용
def convertSequenceToDict(list):
    dict = {}
    argList = range(1, len(list) + 1)
    for k, v in zip(argList, list):
        dict[str(k)] = v
    return dict

# 데이터프레임에 저장된 데이터를 Oracle 테이블로 입력(insert)
cols = [k for k in air_df.dtypes.index]
colnames = ','.join(cols)
colpos = ', '.join([':' + str(i + 1) for i, f in enumerate(cols)])
insert_sql = 'INSERT INTO %s (%s) VALUES (%s)' % (src_table, colnames, colpos)

# INSERT INTO ... VALUES (:1, :2, ...)의 바인드 변수 값을 저장하는 Dictionary 구조 생성
data = [convertSequenceToDict(rec) for rec in air_df.values]

# 바인드 변수와 Dictionary 데이터구조를 활용하여 Bulk Insertion 구현
cur.executemany(insert_sql, data)
conn_ora.commit()

TypeError: expecting number

In [ ]:
# Oracle DB 연결
# 접속정보(connection string) : ID/PASS@CONNECTION_ALIAS
# CONNECTION_ALIAS : Oracle TNSNAMES.ORA 파일에 있는 접속정보 별칭(ALIAS)
conn_ora = cx_Oracle.connect("hr/hr@xe")

# DB 커서(Cursor) 선언
cur = conn_ora.cursor()

# 사용할 Oracle 소스 테이블명 지정
src_table = "Air_df"

# 데이터프레임(rawData)에 저장된 데이터를 Oracle 테이블(d_base3_2)에 입력하기 위한 로직
# d_base3_2 테이블 존재하는지 체크하는 함수
def table_exists(name=None, con=None):
    sql = "select table_name from user_tables where table_name='MYTABLE'".replace('MYTABLE', name.upper())
    df = pd.read_sql(sql, con)

    # 테이블이 존재하면 True, 그렇지 않으면 False 반환
    exists = True if len(df) > 0 else False
    return exists
#
#'요일','공항','상대공항','항공사','편명','등록기호','출도착',
#'부정기타입','계획시각','실제시각','지연여부','지연사유','결항여부','결항사유'
#
# 테이블(d_base3_2) 생성 (테이블이 이미 존재한다면 TRUNCATE TABLE)
if table_exists(src_table, conn_ora):
    cur.execute("TRUNCATE TABLE " + src_table)
else:
    cur.execute("create table " + src_table + " ( \
               Date varchar2(20), \
               요일 varchar2(10),\
               공항 varchar2(20),\
               상대공항 varchar2(20),\
               항공사 varchar2(20),\
               편명 varchar2(20),\
               등록기호 varchar2(100),\
               출도착 varchar2(10),\
               부정기타입 varchar2(10),\
               계획시각 varchar2(100),\
               실제시각 varchar2(100),\
               지연여부 varchar2(10),\
               지연사유 varchar2(20),\
               결항여부 varchar2(10),\
               결항사유 varchar2(20))")

# Sequence 구조를 Dictionary 구조((element, value))로 변환하는 함수
# 예: ("Matt", 1) -> {'1':'Matt', '2':1}
# INSERT INTO ... VALUES (:1, :2, ...) 에서 바인드 변수값을 주기위해 Dictionary item 구조 사용
def convertSequenceToDict(list):
    dict = {}
    argList = range(1, len(list) + 1)
    for k, v in zip(argList, list):
        dict[str(k)] = v
    return dict

# 데이터프레임에 저장된 데이터를 Oracle 테이블로 입력(insert)
cols = [k for k in Air_df.dtypes.index]
colnames = ','.join(cols)
colpos = ', '.join([':' + str(i + 1) for i, f in enumerate(cols)])
insert_sql = 'INSERT INTO %s (%s) VALUES (%s)' % (src_table, colnames, colpos)

# INSERT INTO ... VALUES (:1, :2, ...)의 바인드 변수 값을 저장하는 Dictionary 구조 생성
data = [convertSequenceToDict(rec) for rec in Air_df.values]

# 바인드 변수와 Dictionary 데이터구조를 활용하여 Bulk Insertion 구현

conn_ora.commit()